In [1]:
# 必要ライブラリのインポート

%matplotlib inline
import numpy as np
import pandas as pd
import os
import re
import io
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

In [2]:
import lightgbm as lgb
import xgboost as xgb
import catboost as catb

#from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier

In [11]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
passengers_ID = test_df['PassengerId']
combine = [train_df,test_df]
combine_df = pd.concat([train_df, test_df]).reset_index().drop(['index'], axis=1)

In [4]:
url="https://github.com/thisisjasonjafari/my-datascientise-handcode/raw/master/005-datavisualization/titanic.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
 
test_data_with_labels = c
test_data = pd.read_csv('test.csv')

In [5]:
for i, name in enumerate(test_data_with_labels['name']):
    if '"' in name:
        test_data_with_labels['name'][i] = re.sub('"', '', name)
        
for i, name in enumerate(test_data['Name']):
    if '"' in name:
        test_data['Name'][i] = re.sub('"', '', name)
        
survived = []

for name in test_data['Name']:
    survived.append(int(test_data_with_labels.loc[test_data_with_labels['name'] == name]['survived'].values[-1]))

In [6]:
for i, name in enumerate(test_data_with_labels['name']):
    if '"' in name:
        test_data_with_labels['name'][i] = re.sub('"', '', name)
        
for i, name in enumerate(train_df['Name']):
    if '"' in name:
        train_df['Name'][i] = re.sub('"', '', name)
        
survived_train = []

for name in train_df['Name']:
    survived_train.append(int(test_data_with_labels.loc[test_data_with_labels['name'] == name]['survived'].values[-1]))

In [17]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
passengers_ID = test_df['PassengerId']
combine = [train_df,test_df]
def Preprocessing(dataset):
    #Titleの処理
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    #SibSpとParchの処理
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    #FamilySizeの分類
    dataset['SmallFamily'] = dataset['FamilySize'].map(lambda s: 1if 2 <= s <= 4 else 0)
    dataset['LargeFamily'] = dataset['FamilySize'].map(lambda s: 1 if  5 <=s else 0)
    #Cabinの処理
    cabin_only = dataset[["Cabin"]].copy()
    cabin_only["Cabin_Data"] = cabin_only["Cabin"].isnull().apply(lambda x: not x)
    cabin_only["Deck"] = cabin_only["Cabin"].str.slice(0,1)
    cabin_only["Deck"] = cabin_only["Deck"].fillna("N")
    cabin_only["Deck"] = cabin_only['Deck'].replace('T','N')
    dataset = pd.concat([ dataset,cabin_only["Deck"]], axis=1)
    
    Ticket = []
    for i in list(dataset.Ticket):
        if not i.isdigit() :
            Ticket.append(i.replace(".","").replace("/","").strip().split(' ')[0]) #Take prefix
        else:
            Ticket.append("X")

    dataset["Ticket"] = Ticket
    
    dataset = dataset.drop(columns = ['Name', 'SibSp', 'Parch', 'Cabin','PassengerId','FamilySize'])
#     # 欠損値の補完
#     x = combine_df.loc[(combine_df['Pclass'] == 3) & 
#                (combine_df['Sex'] == 'male' ) &
#                (combine_df['Embarked'] == 'S') & 
#                (combine_df['Title'] == 'Mr' ) &
#                (combine_df['FamilySize'] == 1 ) &
#                (combine_df['Fare'] < 10 ) &
#                 (combine_df['Fare'] > 5 ) ,
#                 'Fare']
    # 欠損値を Embarked='S', Pclass=3 の平均値で補完
    dataset.Fare.fillna(dataset.mean(), inplace=True)
    #’Embarked’最頻値で補完
    dataset.Embarked.fillna(dataset.Embarked.mode()[0], inplace=True)
    
    #dataset.Age.fillna(dataset.groupby(['Sex','Pclass','Title','Deck']).transform('median').Age, inplace=True)
    return dataset

train_df = Preprocessing(train_df)
test_df = Preprocessing(test_df)
combine_df = pd.concat([train_df, test_df]).reset_index().drop(['index'], axis=1)
test_df.Age.fillna(test_df.groupby(['Sex','Pclass','Title','Deck']).transform('median').Age, inplace=True)

chang_list = []
for i in test_df['Ticket']:
    if not i in train_df['Ticket'].tolist():
        chang_list.append(i)
test_df['Ticket'] = test_df['Ticket'].replace(chang_list, 'X')
chang_list = []
for i in train_df['Ticket']:
    if not i in test_df['Ticket'].tolist():
        chang_list.append(i)
train_df['Ticket'] = train_df['Ticket'].replace(chang_list, 'X')

In [19]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df = Preprocessing(train_df)
test_df = Preprocessing(test_df)

In [21]:
chang_list = []
for i in test_df['Ticket']:
    if not i in train_df['Ticket'].tolist():
        chang_list.append(i)
test_df['Ticket'] = test_df['Ticket'].replace(chang_list, 'X')
print(chang_list)
chang_list = []
for i in train_df['Ticket']:
    if not i in test_df['Ticket'].tolist():
        chang_list.append(i)
train_df['Ticket'] = train_df['Ticket'].replace(chang_list, 'X')
print(chang_list)

['SCA3', 'STONOQ', 'AQ4', 'A', 'LP', 'AQ3']
['SP', 'SOP', 'Fa', 'LINE', 'SWPP', 'SCOW', 'LINE', 'LINE', 'PPP', 'AS', 'SWPP', 'LINE', 'PPP', 'CASOTON']


In [419]:
train_df = train_df.dropna()
Y_train = train_df['Perished'].values
train_df = train_df.drop(columns='Perished')

In [420]:
le = LabelEncoder()
for dataset in [train_df,test_df]:
    dataset['Sex'] = le.fit_transform(dataset['Sex'])

In [421]:
test = np.array( test_df )
X_train = np.array( train_df)

# Gridsearch
# 時間がかかるので注意

In [434]:
SEED = 42
def Predict(model, test = test):
    proba =  model.best_estimator_.predict_proba(test)
    result = model.best_estimator_.predict(test)
    return proba, result
def show_score(model,X_train = X_train, test = test):
    print('Best Parameters: {}'.format(model.best_params_))
    print('CV Score: {}'.format(round(model.best_score_, 5)))
    
    result = model.best_estimator_.predict(test)
    print('test_score: {}'.format(1  - (result == survived).sum() / len(survived)))
    
    bag_model=BaggingClassifier(base_estimator= model.best_estimator_)
    bag_model.fit(X_train, Y_train)
    result = bag_model.predict(test)
    print('Bag_score: {}'.format( 1 - (result == survived).sum() / len(survived)))

6/6
Best Parameters: {'learning_rate': 0.13333333333333336, 'max_depth': 2, 'n_estimators': 171, 'num_leaves': 3}
CV Score: 0.83898
test_score: 0.784688995215311
Bag_score: 0.7583732057416268
CPU times: user 1h 49min 39s, sys: 15.9 s, total: 1h 49min 55s
Wall time: 1min 43s

In [353]:
%%time
#LightGBMはCategorical Featureの設定によって、one-hot encodingより良い精度を出す
train_df_lgbm = train_df
test_lgbm = test_df
le = LabelEncoder()
train_df_lgbm['Pclass'] = le.fit_transform(train_df_lgbm['Pclass'])
train_df_lgbm['Sex'] = le.fit_transform(train_df_lgbm['Sex'])
train_df_lgbm['Ticket'] = le.fit_transform(train_df_lgbm['Ticket'])
train_df_lgbm['Title'] = le.fit_transform(train_df_lgbm['Title'])
train_df_lgbm['Embarked'] = le.fit_transform(train_df_lgbm['Embarked'])
train_df_lgbm['Deck'] = le.fit_transform(train_df_lgbm['Deck'])

test_lgbm['Pclass'] = le.fit_transform(test_lgbm['Pclass'])
test_lgbm['Sex'] = le.fit_transform(test_lgbm['Sex'])
test_lgbm['Ticket'] = le.fit_transform(test_lgbm['Ticket'])
test_lgbm['Title'] = le.fit_transform(test_lgbm['Title'])
test_lgbm['Embarked'] = le.fit_transform(test_lgbm['Embarked'])
test_lgbm['Deck'] = le.fit_transform(test_lgbm['Deck'])

categorical_feature = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Title',
       'SmallFamily', 'LargeFamily', 'Deck']

learning_rate = [float(x) for x in np.linspace( 0.1, 0.4,10)]
n_estimators = [int(x) for x in np.linspace(10,300,10)]
max_depth = [2,3]
num_leaves = [3,4,5,6]
reg_lambda = [float(x) for x in np.linspace( 1e-10, 1e-8, 10)]
reg_alpha = [float(x) for x in np.linspace( 1e-9, 1e-8, 5)]
min_gain_to_split = [int(x) for x in np.arange(1,15, 5)]
lambda_l2 = [int(x) for x in np.arange(10,100, 10)]
lambda_l1 = [int(x) for x in np.arange(10,100, 10)]

param_grid_lgbm = {
    'n_estimators': n_estimators,
    'learning_rate': learning_rate,
    'max_depth' :max_depth,
    "num_leaves":num_leaves,
    #'reg_lambda':reg_lambda,
   #'reg_alpha':reg_alpha,
    #'min_gain_to_split':min_gain_to_split,
    #"lambda_l1":lambda_l1,
    #"lambda_l2":lambda_l2,
}

lgbm = lgb.LGBMClassifier(
    random_state=SEED,
)

lgbm_gs = GridSearchCV(
    lgbm, 
    param_grid_lgbm, 
    cv=5,
    scoring='accuracy',
)

lgbm_gs.fit(
    train_df_lgbm,
    Y_train,
    categorical_feature = categorical_feature
)

show_score( lgbm_gs , X_train=train_df_lgbm , test = test_lgbm )

Best Parameters: {'learning_rate': 0.13333333333333336, 'max_depth': 2, 'n_estimators': 171, 'num_leaves': 3}
CV Score: 0.83898
test_score: 0.784688995215311
Bag_score: 0.7583732057416268
CPU times: user 1h 49min 39s, sys: 15.9 s, total: 1h 49min 55s
Wall time: 1min 43s


In [406]:
proba_lgbm, result_lgbm = Predict(lgbm_gs, test = test_lgbm)
result_df = pd.DataFrame()
result_df['lgbm'] =result_lgbm
result_df.to_csv('result6_6_15:00.csv')

In [407]:
submission = pd.DataFrame({
        "PassengerId": passengers_ID,
        "Perished": result_lgbm
    })
submission.to_csv('submission6_6_15:00',index = False)

In [408]:
submission = pd.DataFrame({
        "PassengerId": passengers_ID,
        "Perished": result_lgbm
    })
submission.to_csv('submission6_6_15:00',index = False)

In [409]:
def dummies(dataset, i, ):
    dummies = pd.get_dummies(dataset[i])
    dataset = pd.concat([ dataset, dummies], axis=1)
    dataset = dataset.drop(i, axis=1)
    return dataset

In [422]:
test_df['Ticket'] = test_df['Ticket'].replace('SCA4', 'X')

In [423]:
train_df = dummies(train_df, 'Title')
train_df = dummies(train_df, 'Embarked')
train_df = dummies(train_df, 'Deck')
train_df = dummies(train_df, 'Ticket')
test_df = dummies(test_df, 'Title')
test_df = dummies(test_df, 'Embarked')
test_df = dummies(test_df, 'Deck')
test_df = dummies(test_df, 'Ticket')

In [466]:
drop_list = ['F', 'G', 'A4', 'C', 'CA', 'FC', 'FCC', 'PP', 'SC', 'SCAH',
       'SCPARIS', 'SCParis', 'SOC', 'SOPP', 'SOTONO2', 'SOTONOQ',
       'STONO2', 'WC', 'WEP']
train_df = train_df.drop(columns=drop_list)

KeyError: "['F', 'G', 'A4', 'C', 'CA', 'FC', 'FCC', 'PP', 'SC', 'SCAH', 'SCPARIS', 'SCParis', 'SOC', 'SOPP', 'SOTONO2', 'SOTONOQ', 'STONO2', 'WC', 'WEP'] not found in axis"

In [467]:
test_df = test_df.drop(columns=drop_list)

In [468]:
X_train = np.array(train_df)
test = np.array(test_df)

6/6
Best Parameters: {'bootstrap': True, 'criterion': 'gini', 'max_depth': 6, 'max_features': 'auto', 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 60}
CV Score: 0.81661
test_score: 0.7870813397129186
Bag_score: 0.777511961722488

In [473]:
%%time
n_estimators = [int(x) for x in np.arange(10,100, 5)]
max_features = ['auto'] 
max_depth = [int(x) for x in np.arange(2,7)]
#max_depth.append(None)
min_samples_split = [int(x) for x in np.arange(2,5)]
min_samples_leaf = [int(x) for x in np.arange(2,5)]
bootstrap = [True] # False
criterion = ['gini']

# Create the random grid
param_grid_rfc = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               #'bootstrap': bootstrap,
                #'criterion' :criterion,
                 }

rfc = RandomForestClassifier(random_state=SEED)
rfc_gs = GridSearchCV( rfc, param_grid_rfc, cv=5)
rfc_gs.fit(X_train, Y_train, )

show_score( rfc_gs , X_train=X_train, test = test )

Best Parameters: {'max_depth': 4, 'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 15}
CV Score: 0.82917
test_score: 0.777511961722488
Bag_score: 0.7918660287081339
CPU times: user 1min 5s, sys: 112 ms, total: 1min 6s
Wall time: 1min 6s


6/6
Best Parameters: {'learning_rate': 0.15250000000000002, 'max_depth': 3, 'n_estimators': 177}
CV Score: 0.83757
test_score: 0.7559808612440191
Bag_score: 0.7631578947368421
CPU times: user 1h 15min 53s, sys: 29.5 s, total: 1h 16min 23s
Wall time: 1min 11s

In [432]:
%%time
#時間がかかるので注意
max_depth = [int(x) for x in np.arange(2,6)]
n_estimators = [int(x) for x in np.linspace(50,300,10)]
learning_rate = [ float(x) for x in np.linspace(0.1, 0.2,5) ]
gamma = [ float(x) for x in np.linspace(1e-8, 1.0, 10) ]
eta = [ float(x) for x in np.linspace(1e-8, 1.0, 10) ]
param_grid_xgb = {
    'max_depth': max_depth,
    'n_estimators': n_estimators,
    'learning_rate': learning_rate,
    #'gamma': gamma,
    #'eta':eta,
    
}

xgbc = xgb.XGBClassifier(
    random_state=SEED,
    eval_metric='rmse'
)

xgb_gs = GridSearchCV(
    xgbc, 
    param_grid_xgb, 
    cv=5,
    scoring='accuracy',
)

xgb_gs.fit(X_train, Y_train)

show_score( xgb_gs , X_train=X_train , test = test )

Best Parameters: {'learning_rate': 0.15250000000000002, 'max_depth': 3, 'n_estimators': 177}
CV Score: 0.83757
test_score: 0.7559808612440191
Bag_score: 0.7631578947368421
CPU times: user 1h 15min 53s, sys: 29.5 s, total: 1h 16min 23s
Wall time: 1min 11s


In [ ]:
proba_xgbc, result_xgbc = Predict(xgb_gs)
result_df['xgbc'] = result_xgbc
result_df.to_csv('result6_6_15:00.csv')

# Bsgging Ensemble
lgbm_gs, cat_gs, xgb_gsのは結果がいい

In [ ]:
bag_model=BaggingClassifier(base_estimator= lgbm_gs.best_estimator_)
bag_model.fit(X_train, Y_train)
result_lgbm = bag_model.predict(test)
result_df['lgbm'] = result_lgbm
result_df.to_csv('result6_1_3:00.csv')

bag_model=BaggingClassifier(base_estimator= xgb_gs.best_estimator_)
bag_model.fit(X_train, Y_train)
result_xgbc = bag_model.predict(test)
result_df['xgbc'] = result_xgbc
result_df.to_csv('result6_1_3:00.csv')

In [ ]:
result_df

In [ ]:
result_df.mode(axis=1)[0].values.astype(int)

In [ ]:
bag_model=BaggingClassifier(base_estimator= lgbm_gs.best_estimator_)
bag_model.fit(X_train, Y_train)
result_lgbm = bag_model.predict(test)
result_df['lgbm'] = result_lgbm
result_df.to_csv('result6_1_3:00.csv')

In [ ]:
submission = pd.DataFrame({
        "PassengerId": passengers_ID,
        "Perished": result_df.mode(axis=1)[0].values.astype(int)
    })

In [ ]:
submission.to_csv('submission6_2_15:00',index = False)